# Algorytm RSVD(k)

Niech $A$ - losowa $M \times N$ o zadanych wartościach szczególnych

Zbadać:

faktyczny błąd RSVD: $||A - U D V^T||_F$ (lub $||\cdot||_2$)

błąd spodziewany

faktyczny błąd wartości szczególnych: $\max_{1 \leq i \leq k} |\sigma_i - d_i|$

oszacowanie błędu faktycznego

In [333]:
import numpy as np

def random_ortogonal_matrix(n):
    A = np.random.randn(n, n)
    Q, _ = np.linalg.qr(A)
    return Q

def random_matrix_with_singular_values(m, n, sigma):
    U = random_ortogonal_matrix(m)
    V = random_ortogonal_matrix(n)
    S = np.diag(sigma)
    S = np.concatenate((S, np.zeros((m - n, n))), axis=0)
    return U @ S @ V

In [334]:
def rsvd(A, k, p, q = 0):
    '''p - oversampling'''
    
    _, N = A.shape
    G = np.random.randn(N, k + p)
    Y = A @ G
    for _ in range(q): # q iterations of power method
        Y = A @ (A.T @ Y)
    
    Q, _ = np.linalg.qr(Y)
    B = Q.T @ A
    U, S, VT = np.linalg.svd(B, full_matrices=False) # shape: (k+p, k+p), (k+p,), (k+p, N)
    U, S, VT = U[:, :k], S[:k], VT[:k, :]
    
    return Q @ U, S, VT

In [335]:
M, N = 100, 20
sigma = 10 * np.random.randn(N)
A = random_matrix_with_singular_values(M, N, sigma)
A

array([[ 0.02618262,  1.33905706, -1.01285918, ..., -1.94674684,
         0.19784233,  0.944064  ],
       [-2.68394954,  0.37833972,  0.3450654 , ...,  0.69903926,
         0.89261298,  0.05110147],
       [ 0.42301405, -0.37895783,  0.53483485, ..., -0.8913327 ,
        -0.75300206,  0.38891861],
       ...,
       [ 1.23788904,  0.39980835,  0.55564557, ...,  1.58510239,
        -0.86161122,  1.86682868],
       [ 0.77656341, -1.60719823, -0.51969627, ..., -0.74373482,
        -0.44498623, -1.874288  ],
       [-1.57218522, -0.0160877 ,  0.17041063, ..., -0.61875594,
        -0.81882852, -1.32085206]])

In [336]:
_, S, _ = np.linalg.svd(A, full_matrices=False)
S

array([30.10148412, 22.3609418 , 20.08499041, 17.59769445, 15.00742635,
       10.53985484,  9.51146017,  9.48065209,  9.01709679,  8.58369158,
        8.4397046 ,  7.69741075,  7.53713323,  6.49376364,  6.36080211,
        5.07891308,  4.50891945,  2.75819621,  2.52978011,  2.11597193])

In [337]:
k, p = 5, 3
U, D, VT = rsvd(A, k, p)
D

array([28.43022977, 19.96663183, 15.68402529, 15.37845556, 12.89156422])

In [338]:
U.shape, D.shape, VT.shape

((100, 5), (5,), (5, 20))

# Oszacowanie błędu faktycznego

Niech $T := A - UDV^T$, gdzie $UDV^T \approx A$ - wynik algorytmu RSVD

Z prawdopodobieństwem $1 - \alpha^r$ zachodzi

$$||T||_2 \leq \frac{1}{\alpha} \sqrt{\frac{2}{\pi}} \max_{i=1..r} ||Tx_i||_2,$$

gdzie $x_i$ - wektory o współczynnikach z rozkładu $\mathcal{N}(0,1)$

In [339]:
def print_errors(A, U, D, VT, S, k, p):
    error_fro = np.linalg.norm(A - U @ np.diag(D) @ VT, ord="fro")
    expected_error_fro = np.sqrt(1 + k / (p - 1)) * np.linalg.norm(S[k:])

    print(f'błąd (frobenius): {error_fro}')
    print(f'oszacowanie oczekiwanego błędu (frobenius): {expected_error_fro}')
    print(f'stosunek błędu do oczekiwanego błędu (frobenius): {error_fro / expected_error_fro}')
    
    error_2 = np.linalg.norm(A - U @ np.diag(D) @ VT, ord=2)
    expected_error_2 = (1 + np.sqrt(k / p)) * S[k] + np.e * np.sqrt(k + p) / p * np.linalg.norm(S[k:])

    print(f'błąd (2): {error_2}')
    print(f'oszacowanie oczekiwanego błędu (2): {expected_error_2}')
    print(f'stosunek błędu do oczekiwanego błędu (2): {error_2 / expected_error_2}')
    
    T = A - U @ np.diag(D) @ VT
    alpha, r = 0.5, 3
    error_estimate = 1 / alpha * np.sqrt(2 / np.pi) * max([np.linalg.norm(T @ np.random.randn(N)) for _ in range(r)])
    print(f'oszacowanie błędu (2): {error_estimate}')
    
    print(f'maksimum błędów wartości szczególnych: {np.max(np.abs(D - S[:k]))}')

In [340]:
print_errors(A, U, D, VT, S, k, p)
D, S[:k]

błąd (frobenius): 35.633560413471365
oszacowanie oczekiwanego błędu (frobenius): 52.28132294637489
stosunek błędu do oczekiwanego błędu (frobenius): 0.6815734263270425
błąd (2): 18.3593969167465
oszacowanie oczekiwanego błędu (2): 95.76616357559865
stosunek błędu do oczekiwanego błędu (2): 0.1917106860217224
oszacowanie błędu (2): 60.48058149747148
maksimum błędów wartości szczególnych: 4.400965125648842


(array([28.43022977, 19.96663183, 15.68402529, 15.37845556, 12.89156422]),
 array([30.10148412, 22.3609418 , 20.08499041, 17.59769445, 15.00742635]))

In [341]:
U, D, VT = rsvd(A, k, p, q = 1)
print_errors(A, U, D, VT, S, k, p)
D, S[:k]

błąd (frobenius): 28.30631388188768
oszacowanie oczekiwanego błędu (frobenius): 52.28132294637489
stosunek błędu do oczekiwanego błędu (frobenius): 0.5414230606008488
błąd (2): 10.721843948203391
oszacowanie oczekiwanego błędu (2): 95.76616357559865
stosunek błędu do oczekiwanego błędu (2): 0.11195858273824943
oszacowanie błędu (2): 55.37639106912689
maksimum błędów wartości szczególnych: 0.3352602895590451


(array([30.07526882, 22.33801575, 19.94598429, 17.26243417, 14.99276093]),
 array([30.10148412, 22.3609418 , 20.08499041, 17.59769445, 15.00742635]))

In [342]:
U, D, VT = rsvd(A, k, p, q = 2)
print_errors(A, U, D, VT, S, k, p)
D, S[:k]

błąd (frobenius): 27.960777354035443
oszacowanie oczekiwanego błędu (frobenius): 52.28132294637489
stosunek błędu do oczekiwanego błędu (frobenius): 0.5348138833960819
błąd (2): 10.540209217968608
oszacowanie oczekiwanego błędu (2): 95.76616357559865
stosunek błędu do oczekiwanego błędu (2): 0.11006193444982343
oszacowanie błędu (2): 49.985038928632555
maksimum błędów wartości szczególnych: 0.01858608830620767


(array([30.10147107, 22.35901052, 20.08306589, 17.59400754, 14.98884027]),
 array([30.10148412, 22.3609418 , 20.08499041, 17.59769445, 15.00742635]))

In [343]:
U, D, VT = rsvd(A, k, p, q = 3)
print_errors(A, U, D, VT, S, k, p)
D, S[:k]

błąd (frobenius): 27.968160828439665
oszacowanie oczekiwanego błędu (frobenius): 52.28132294637489
stosunek błędu do oczekiwanego błędu (frobenius): 0.5349551092486066
błąd (2): 10.540377466615302
oszacowanie oczekiwanego błędu (2): 95.76616357559865
stosunek błędu do oczekiwanego błędu (2): 0.1100636913192689
oszacowanie błędu (2): 49.91441454566886
maksimum błędów wartości szczególnych: 0.040112235136090746


(array([30.1014824 , 22.36083966, 20.08495312, 17.59609913, 14.96731412]),
 array([30.10148412, 22.3609418 , 20.08499041, 17.59769445, 15.00742635]))

Do pomyślenia: jak zrobić tak, żeby policzyć `rsvd(k+m)` znając `rsvd(k)`